In [ ]:
!pip install uv
!uv pip install -r  requirements.txt
!uv pip install streamlit
!uv pip install -U ipywidgets
!uv pip install shap snowflake-ml-python==1.19.0
!uv pip install folium
!uv pip install geopy

In [ ]:
%%sql -r dataframe_1
select current_role();


    

In [ ]:
#Update this VERSION_NUM to version your features, models etc!
VERSION_NUM = '0'
DB = "EY_DATA_CHALLENGE" 
SCHEMA = "DATA_SCHEMA" 
ROLE ="ACCOUNTADMIN"

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import math
import pickle
import shap
from datetime import datetime
import streamlit as st
from xgboost import XGBClassifier

# Snowpark ML
from snowflake.ml.registry import Registry
from snowflake.ml.modeling.tune import get_tuner_context
from snowflake.ml.modeling import tune
from entities import search_algorithm

#Snowflake feature store
from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode

# Snowpark session
from snowflake.snowpark import DataFrame
from snowflake.snowpark.functions import col, to_timestamp, min, max, month, dayofweek, dayofyear, avg, date_add, sql_expr,year,quarter,date_trunc
from snowflake.snowpark.types import IntegerType
from snowflake.snowpark import Window

#setup snowpark session
from snowflake.snowpark.context import get_active_session
session = get_active_session()

session.use_database(DB)
session.use_schema(SCHEMA)
session

In [ ]:
print("Reading table data...")
df_waterquality = session.table("WATERQUALITY_TRAINING")
df_waterquality.show(5)
df_waterquality.select(min("SAMPLE_DATE"),max("SAMPLE_DATE"))


### Feature Engineering with Snowpark APIs

In [ ]:
#Create a dict with keys for feature names and values containing transform code

feature_eng_dict = dict()

#Timstamp features
feature_eng_dict["MONTH"] = month("SAMPLE_DATE")
feature_eng_dict["QUARTER"] = quarter("SAMPLE_DATE") 
feature_eng_dict["YEAR"] = year("SAMPLE_DATE") 
feature_eng_dict["QUARTER_DATE"] = date_trunc("quarter", col("SAMPLE_DATE"))

year_qtr_partition = Window.partition_by("YEAR","QUARTER").order_by("YEAR")
feature_eng_dict["AVG_YEAR_QTR_ALKALINITY"] = avg("TOTAL_ALKALINITY").over(year_qtr_partition)

feature_eng_dict["PRECEDING_AVG_YEAR_QTR_ALKALINITY"] = sql_expr(""" 
            AVG("TOTAL_ALKALINITY") OVER( PARTITION BY  "YEAR","QUARTER" ORDER BY  "SAMPLE_DATE" )
    """ )

df_waterquality = df_waterquality.with_columns(feature_eng_dict.keys(), feature_eng_dict.values())
df_waterquality.show(3)
df_waterquality.filter((df_waterquality["YEAR"]== 2012) & (df_waterquality["QUARTER"]== 2)).show(3)

In [ ]:
%%sql -r dataframe_2

SELECT
    LATITUDE,
    DATE_PART(year, SAMPLE_DATE) as YEAR,
    DATE_PART(quarter, SAMPLE_DATE) as QUARTER,
    AVG(TOTAL_ALKALINITY) as AVG_ALKALINITY,
    LAG(AVG(TOTAL_ALKALINITY), 1) OVER (PARTITION BY LATITUDE ORDER BY DATE_PART(year, SAMPLE_DATE), DATE_PART(quarter, SAMPLE_DATE)) AS PREV_QUARTER_AVG,
    CASE 
        WHEN LAG(AVG(TOTAL_ALKALINITY), 1) OVER (PARTITION BY LATITUDE ORDER BY DATE_PART(year, SAMPLE_DATE), DATE_PART(quarter, SAMPLE_DATE)) IS NULL THEN ''
        WHEN LAG(AVG(TOTAL_ALKALINITY), 1) OVER (PARTITION BY LATITUDE ORDER BY DATE_PART(year, SAMPLE_DATE), DATE_PART(quarter, SAMPLE_DATE)) > AVG(TOTAL_ALKALINITY) THEN 'GETTING WORSE'
        ELSE 'GETTING BETTER'
    END AS TREND
FROM WATERQUALITY_TRAINING
where LATITUDE = -34.405833
GROUP BY LATITUDE, YEAR, QUARTER  
ORDER BY LATITUDE, YEAR, QUARTER  ;



In [ ]:
import matplotlib.pyplot as plt
LATITUDE = -34.405833
# Convert Snowpark DataFrame to pandas
df_pandas  = df_waterquality.filter(col("LATITUDE") == LATITUDE).select("QUARTER_DATE", "AVG_YEAR_QTR_ALKALINITY").sort("QUARTER_DATE").to_pandas()

plt.figure(figsize=(12, 6))
plt.plot(df_pandas["QUARTER_DATE"], df_pandas["AVG_YEAR_QTR_ALKALINITY"], marker='o')
plt.xlabel('Quarter')
plt.ylabel('Average Alkalinity')
plt.title(f"Average Alkalinity Over Time for Latitude  {LATITUDE}")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:

# if SKIP:
# from geopy.geocoders import Nominatim
# import time

# geolocator = Nominatim(user_agent="water_quality_checker")

# all_coords = df_waterquality.select("LATITUDE", "LONGITUDE").distinct().to_pandas()

# print("Identifying locations (checking first 10 coordinates):\n")
# locations_found = []

# for idx, row in all_coords.head(10).iterrows():
#     try:
#         location = geolocator.reverse(f"{row['LATITUDE']}, {row['LONGITUDE']}", language='en')
#         address = location.raw.get('address', {})
        
#         # Extract relevant info
#         water_body = address.get('water', 'Unknown')
#         town = address.get('town', address.get('city', address.get('village', 'Unknown')))
#         state = address.get('state', 'Unknown')
        
#         print(f"Point {idx+1}: ({row['LATITUDE']:.4f}, {row['LONGITUDE']:.4f})")
#         print(f"  Location: {town}, {state}")
#         print(f"  Water body: {water_body}")
#         print(f"  Full address: {location.address}")
#         print()
        
#         locations_found.append({
#             'lat': row['LATITUDE'],
#             'lon': row['LONGITUDE'],
#             'water_body': water_body,
#             'town': town,
#             'state': state
#         })
        
#         time.sleep(1.5)  # Respect API rate limits
#     except Exception as e:
#         print(f"Could not geocode: ({row['LATITUDE']}, {row['LONGITUDE']})")
#         print(f"  Error: {e}\n")

# Summary
# import pandas as pd
# if locations_found:
#     summary_df = pd.DataFrame(locations_found)
#     print("\n=== SUMMARY ===")
#     print(f"Water bodies found: {summary_df['water_body'].unique()}")
#     print(f"Towns/cities: {summary_df['town'].unique()}")

In [ ]:
all_coords = df_waterquality.select("LATITUDE", "LONGITUDE").distinct().to_pandas()

print(f"Your data coordinates:")
print(f"  Latitude: {all_coords['LATITUDE'].min():.4f} to {all_coords['LATITUDE'].max():.4f}")
print(f"  Longitude: {all_coords['LONGITUDE'].min():.4f} to {all_coords['LONGITUDE'].max():.4f}")

print("\n" + "=" * 80)
print("KNOWN SOUTH AFRICAN RIVER APPROXIMATE RANGES:")
print("=" * 80)

rivers = {
    "Vaal River": {"lat": (-28.5, -26.0), "lon": (24.0, 29.0)},
    "Orange River": {"lat": (-29.0, -26.5), "lon": (16.5, 29.0)},
    "Limpopo River": {"lat": (-25.5, -22.0), "lon": (27.0, 32.0)},
    "Tugela River": {"lat": (-29.5, -28.0), "lon": (29.0, 31.0)},
    "Olifants River": {"lat": (-25.5, -23.5), "lon": (30.0, 32.0)},
}

# Check which rivers your coordinates might match
lat_min, lat_max = all_coords['LATITUDE'].min(), all_coords['LATITUDE'].max()
lon_min, lon_max = all_coords['LONGITUDE'].min(), all_coords['LONGITUDE'].max()

print("\nPossible matches based on coordinate ranges:\n")
for river_name, bounds in rivers.items():
    lat_overlap = not (lat_max < bounds['lat'][0] or lat_min > bounds['lat'][1])
    lon_overlap = not (lon_max < bounds['lon'][0] or lon_min > bounds['lon'][1])
    
    if lat_overlap and lon_overlap:
        print(f"✓ {river_name}")
        print(f"  River range: Lat {bounds['lat']}, Lon {bounds['lon']}")
    else:
        print(f"✗ {river_name} - coordinates don't match")

print("\n" + "=" * 80)
print("RECOMMENDATION: Click the Google Maps links above to visually confirm")
print("=" * 80)

In [ ]:
import matplotlib.pyplot as plt

# Get coordinates
vaal_coords = df_waterquality.select("LATITUDE", "LONGITUDE").distinct().to_pandas()
vaal_coords_sorted = vaal_coords.sort_values('LONGITUDE')

plt.figure(figsize=(12, 8))
plt.scatter(vaal_coords_sorted['LONGITUDE'], vaal_coords_sorted['LATITUDE'], 
            c='blue', s=100, alpha=0.6, edgecolors='black')
plt.plot(vaal_coords_sorted['LONGITUDE'], vaal_coords_sorted['LATITUDE'], 
         'b-', alpha=0.3, linewidth=2)

plt.xlabel('Longitude', fontsize=12)
plt.ylabel('Latitude', fontsize=12)
plt.title('Vaal River Water Quality Sample Locations', fontsize=14)
plt.grid(True, alpha=0.3)

# Add labels for each point
for idx, row in vaal_coords_sorted.iterrows():
    plt.annotate(f"({row['LATITUDE']:.2f}, {row['LONGITUDE']:.2f})", 
                 (row['LONGITUDE'], row['LATITUDE']),
                 fontsize=8, alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Get coordinates
vaal_coords = df_waterquality.select("LATITUDE", "LONGITUDE").distinct().to_pandas()
vaal_coords_sorted = vaal_coords.sort_values('LONGITUDE')

fig, ax = plt.subplots(figsize=(14, 10))

# Plot the river path
ax.plot(vaal_coords_sorted['LONGITUDE'], vaal_coords_sorted['LATITUDE'], 
        'b-', alpha=0.4, linewidth=3, label='River Path')

# Plot sample points
scatter = ax.scatter(vaal_coords_sorted['LONGITUDE'], vaal_coords_sorted['LATITUDE'], 
                     c=range(len(vaal_coords_sorted)), cmap='viridis', 
                     s=150, alpha=0.8, edgecolors='black', linewidth=1.5,
                     label='Sample Locations')

# Add colorbar to show progression
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Sample Location Index', rotation=270, labelpad=20)

ax.set_xlabel('Longitude (East →)', fontsize=13, fontweight='bold')
ax.set_ylabel('Latitude (North ↑)', fontsize=13, fontweight='bold')
ax.set_title('Water Quality Sample Locations\nSouth Africa', fontsize=15, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--')
ax.legend(loc='best', fontsize=11)

# Add aspect ratio to make it more map-like
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

print(f"\nTotal sample locations: {len(vaal_coords)}")
print(f"Latitude range: {vaal_coords['LATITUDE'].min():.4f} to {vaal_coords['LATITUDE'].max():.4f}")
print(f"Longitude range: {vaal_coords['LONGITUDE'].min():.4f} to {vaal_coords['LONGITUDE'].max():.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Get coordinates with average alkalinity
vaal_data = df_waterquality.group_by("LATITUDE", "LONGITUDE").agg(
    avg("TOTAL_ALKALINITY").alias("AVG_ALKALINITY")
).to_pandas()

vaal_data_sorted = vaal_data.sort_values('LONGITUDE')

fig, ax = plt.subplots(figsize=(14, 10))

# Color points by alkalinity level
scatter = ax.scatter(vaal_data_sorted['LONGITUDE'], vaal_data_sorted['LATITUDE'], 
                     c=vaal_data_sorted['AVG_ALKALINITY'], cmap='RdYlGn_r', 
                     s=200, alpha=0.8, edgecolors='black', linewidth=1.5)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Average Total Alkalinity', rotation=270, labelpad=20, fontsize=11)

# Connect the dots
ax.plot(vaal_data_sorted['LONGITUDE'], vaal_data_sorted['LATITUDE'], 
        'gray', alpha=0.3, linewidth=2, linestyle='--')

ax.set_xlabel('Longitude (East →)', fontsize=13, fontweight='bold')
ax.set_ylabel('Latitude (North ↑)', fontsize=13, fontweight='bold')
ax.set_title('Water Quality\nColored by Average Alkalinity', fontsize=15, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

vaal_coords = df_waterquality.select("LATITUDE", "LONGITUDE").distinct().to_pandas()
vaal_coords_sorted = vaal_coords.sort_values('LONGITUDE').reset_index(drop=True)

fig, ax = plt.subplots(figsize=(14, 10))

ax.scatter(vaal_coords_sorted['LONGITUDE'], vaal_coords_sorted['LATITUDE'], 
           c='blue', s=150, alpha=0.7, edgecolors='black', linewidth=1.5)

ax.plot(vaal_coords_sorted['LONGITUDE'], vaal_coords_sorted['LATITUDE'], 
        'b-', alpha=0.3, linewidth=2)

# Add point numbers
for idx, row in vaal_coords_sorted.iterrows():
    ax.annotate(f"{idx+1}", 
                (row['LONGITUDE'], row['LATITUDE']),
                fontsize=9, fontweight='bold', 
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
                ha='center')

ax.set_xlabel('Longitude', fontsize=13, fontweight='bold')
ax.set_ylabel('Latitude', fontsize=13, fontweight='bold')
ax.set_title('Sample Locations (Numbered)', fontsize=15, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

# Print coordinate list
print("\nLocation Details:")
for idx, row in vaal_coords_sorted.iterrows():
    print(f"Point {idx+1}: Lat {row['LATITUDE']:.4f}, Lon {row['LONGITUDE']:.4f}")

In [ ]:
%%sql -r dataframe_6
select case
     when LATITUDE between -29.0 and -26.5 and LONGITUDE between 16.5 and 29.0  then 'Orange River'
     when LATITUDE between -28.5 and -26.0 and LONGITUDE between 24.0 and 29.0  then 'Vaal River'
     when LATITUDE between -25.5 and -22.0 and LONGITUDE between 27.0 and 32.0  then 'Limpopo River'
     when LATITUDE between -29.5 and -28.0 and LONGITUDE between 29.0 and 31.0  then 'Tugela River'
     when LATITUDE between -25.5 and -23.5 and LONGITUDE between 30.0 and 32.0  then 'Olifans River'
     else 'Other'
     end as LOCATION,
     count(*) as measurement_count
     from LANDSAT_TRAINING
      group by all
     order by LOCATION,measurement_count DESC;


In [ ]:
%%sql -r dataframe_7
select distinct(LATITUDE),LONGITUDE from (select case
                                                     when LATITUDE between -29.0 and -26.5 and LONGITUDE between 16.5 and 29.0  then 'Orange River'
                                                     when LATITUDE between -28.5 and -26.0 and LONGITUDE between 24.0 and 29.0  then 'Vaal River'
                                                     when LATITUDE between -25.5 and -22.0 and LONGITUDE between 27.0 and 32.0  then 'Limpopo River'
                                                     when LATITUDE between -29.5 and -28.0 and LONGITUDE between 29.0 and 31.0  then 'Tugela River'
                                                     when LATITUDE between -25.5 and -23.5 and LONGITUDE between 30.0 and 32.0  then 'Olifans River'
                                                     else 'Other'
                                                     end as LOCATION,LATITUDE,LONGITUDE
                                                  from LANDSAT_TRAINING ) subquery
                                                  where LOCATION='Other';


In [ ]:
## Analysis of "Other" Coordinates
import pandas as pd

# Your unmapped coordinates
unmapped = pd.DataFrame({
    'LATITUDE': [-32.502778, -30.679722, -25.99136, -31.104278, -29.803611, -25.81048333, 
                 -30.77586667, -34.092222, -29.042222, -33.970833, -30.830556, -30.160091, 
                 -34.039722, -29.987778, -34.031944, -28.338333, -34.075556, -31.436944, 
                 -34.250556, -25.513889, -26.006389, -33.897778, -25.67386, -28.44555556, 
                 -31.5653, -28.068333, -30.336111, -27.5275, -30.13573889, -32.601389, 
                 -33.980556, -31.860278, -33.72266667, -33.501667, -32.595833, -33.31388889, 
                 -33.13083, -29.758889, -28.801278, -29.033333, -34.02861, -29.1825, 
                 -33.464444, -31.554722, -25.681111, -31.031389, -28.747778, -26.905, 
                 -27.937778, -29.615556, -33.08883333, -28.308889, -29.161667, -30.86013889, 
                 -29.658056, -31.396, -33.38056, -30.570833, -34.065833, -27.363611, 
                 -27.169444, -26.03611, -33.870833, -33.818056, -34.329722, -29.651389, 
                 -34.405833, -29.160278, -30.534167, -33.7075, -25.57877778, -29.140556, 
                 -30.398056, -29.641944, -27.602222, -30.27402, -29.028629, -34.005833, 
                 -25.73411, -31.000833, -31.394444, -32.94583, -30.568333, -29.7775, 
                 -29.744167, -30.733889],
    'LONGITUDE': [19.535833, 26.7125, 27.84208, 29.399722, 30.516111, 27.90955222, 
                  29.03221667, 21.295278, 24.6, 22.548333, 26.921389, 27.40124, 
                  22.133333, 29.851667, 22.053333, 31.373611, 20.145556, 20.474722, 
                  20.9925, 31.224444, 29.253889, 20.012778, 31.57527, 32.325, 
                  18.3306, 31.55, 27.359444, 30.858056, 30.67422222, 18.750556, 
                  21.653333, 18.6875, 18.61691667, 21.624167, 19.009444, 19.07472222, 
                  18.86278, 30.935278, 31.955389, 23.983333, 22.22222, 23.575556, 
                  18.740556, 29.245556, 31.783333, 28.883333, 31.745833, 32.324722, 
                  31.209444, 27.065278, 19.21878333, 31.9025, 23.696389, 29.07125, 
                  25.973333, 29.2651, 19.30167, 29.150556, 20.404167, 31.783333, 
                  29.233889, 30.99778, 20.003333, 19.694722, 18.990278, 22.746389, 
                  19.600556, 26.573333, 24.961944, 18.97444444, 29.12747222, 31.391944, 
                  30.601944, 30.6875, 29.942778, 30.69602, 24.638389, 22.351111, 
                  27.21422, 26.353056, 20.947222, 18.33667, 29.422778, 29.470833, 
                  29.905833, 29.828333]
})

# Classify into river systems
def classify_river(lat, lon):
    # Breede River (Western Cape)
    if -34.5 < lat < -33.5 and 18.5 < lon < 20.5:
        return 'Breede River'
    
    # Olifants River (Western Cape - different from Limpopo Olifants)
    if -32.8 < lat < -31.0 and 18.0 < lon < 19.5:
        return 'Olifants River (Western Cape)'
    
    # Berg River (Western Cape)
    if -33.8 < lat < -33.0 and 18.3 < lon < 19.0:
        return 'Berg River'
    
    # Gouritz River (Western Cape)
    if -34.5 < lat < -33.5 and 20.5 < lon < 22.5:
        return 'Gouritz River'
    
    # Great Fish River (Eastern Cape)
    if -34.0 < lat < -32.0 and 25.5 < lon < 27.5:
        return 'Great Fish River'
    
    # Sundays River (Eastern Cape)
    if -34.0 < lat < -32.5 and 24.5 < lon < 26.0:
        return 'Sundays River'
    
    # Umzimvubu River (Eastern Cape)
    if -31.5 < lat < -30.5 and 28.5 < lon < 29.5:
        return 'Umzimvubu River'
    
    # Mzimkulu River (KwaZulu-Natal)
    if -31.0 < lat < -30.0 and 29.5 < lon < 30.5:
        return 'Mzimkulu River'
    
    # Umfolozi River (KwaZulu-Natal)
    if -28.5 < lat < -27.5 and 31.5 < lon < 32.5:
        return 'Umfolozi River'
    
    # Pongola River (KwaZulu-Natal)
    if -27.5 < lat < -26.5 and 31.5 < lon < 32.5:
        return 'Pongola River'
    
    # Komati River (Mpumalanga)
    if -26.0 < lat < -25.0 and 31.0 < lon < 32.0:
        return 'Komati River'
    
    # Crocodile River (Mpumalanga)
    if -26.0 < lat < -25.5 and 30.5 < lon < 31.5:
        return 'Crocodile River (East)'
    
    # Inkomati tributaries
    if -25.5 < lat < -25.0 and 31.0 < lon < 32.0:
        return 'Inkomati River System'
    
    # Thukela/Tugela extended
    if -29.5 < lat < -28.0 and 29.5 < lon < 31.5:
        return 'Tugela River (Extended)'
    
    # Buffalo River (Eastern Cape)
    if -33.0 < lat < -32.5 and 27.0 < lon < 28.0:
        return 'Buffalo River'
    
    # Mgeni River (KwaZulu-Natal)
    if -30.0 < lat < -29.5 and 30.5 < lon < 31.0:
        return 'Mgeni River'
    
    # Mooi River (KwaZulu-Natal)
    if -29.5 < lat < -29.0 and 30.0 < lon < 31.0:
        return 'Mooi River'
    
    # Caledon River (Free State/Lesotho border)
    if -30.0 < lat < -28.5 and 27.0 < lon < 28.5:
        return 'Caledon River'
    
    # Modder River (Free State)
    if -29.5 < lat < -29.0 and 25.5 < lon < 26.5:
        return 'Modder River'
    
    # Riet River (Free State)
    if -30.0 < lat < -29.0 and 24.5 < lon < 26.0:
        return 'Riet River'
    
    # Harts River (Northern Cape)
    if -28.5 < lat < -27.5 and 24.0 < lon < 25.0:
        return 'Harts River'
    
    return 'Unknown/Coastal'

unmapped['RIVER_SYSTEM'] = unmapped.apply(lambda row: classify_river(row['LATITUDE'], row['LONGITUDE']), axis=1)

# Summary
print("=" * 80)
print("RIVER CLASSIFICATION SUMMARY")
print("=" * 80)
print(unmapped.groupby('RIVER_SYSTEM').size().sort_values(ascending=False))
print(f"\nTotal unmapped points: {len(unmapped)}")

In [ ]:
%%sql -r dataframe_4
CREATE OR REPLACE TABLE EY_DATA_CHALLENGE.DATA_SCHEMA.RIVER_LOCATIONS AS
SELECT 
    LATITUDE,
    LONGITUDE,
    CASE
        -- Original rivers
        WHEN LATITUDE BETWEEN -29.0 AND -26.5 AND LONGITUDE BETWEEN 16.5 AND 29.0 THEN 'Orange River'
        WHEN LATITUDE BETWEEN -28.5 AND -26.0 AND LONGITUDE BETWEEN 24.0 AND 29.0 THEN 'Vaal River'
        WHEN LATITUDE BETWEEN -25.5 AND -22.0 AND LONGITUDE BETWEEN 27.0 AND 32.0 THEN 'Limpopo River'
        WHEN LATITUDE BETWEEN -29.5 AND -28.0 AND LONGITUDE BETWEEN 29.0 AND 31.0 THEN 'Tugela River'
        WHEN LATITUDE BETWEEN -25.5 AND -23.5 AND LONGITUDE BETWEEN 30.0 AND 32.0 THEN 'Olifants River (Limpopo)'
        
        -- Western Cape rivers
        WHEN LATITUDE BETWEEN -34.5 AND -33.5 AND LONGITUDE BETWEEN 18.5 AND 20.5 THEN 'Breede River'
        WHEN LATITUDE BETWEEN -32.8 AND -31.0 AND LONGITUDE BETWEEN 18.0 AND 19.5 THEN 'Olifants River (Western Cape)'
        WHEN LATITUDE BETWEEN -33.8 AND -33.0 AND LONGITUDE BETWEEN 18.3 AND 19.0 THEN 'Berg River'
        WHEN LATITUDE BETWEEN -34.5 AND -33.5 AND LONGITUDE BETWEEN 20.5 AND 22.5 THEN 'Gouritz River'
        
        -- Eastern Cape rivers
        WHEN LATITUDE BETWEEN -34.0 AND -32.0 AND LONGITUDE BETWEEN 25.5 AND 27.5 THEN 'Great Fish River'
        WHEN LATITUDE BETWEEN -34.0 AND -32.5 AND LONGITUDE BETWEEN 24.5 AND 26.0 THEN 'Sundays River'
        WHEN LATITUDE BETWEEN -31.5 AND -30.5 AND LONGITUDE BETWEEN 28.5 AND 29.5 THEN 'Umzimvubu River'
        
        -- KwaZulu-Natal rivers
        WHEN LATITUDE BETWEEN -31.0 AND -30.0 AND LONGITUDE BETWEEN 29.5 AND 30.5 THEN 'Mzimkulu River'
        WHEN LATITUDE BETWEEN -30.0 AND -29.5 AND LONGITUDE BETWEEN 30.5 AND 31.0 THEN 'Mgeni River'
        WHEN LATITUDE BETWEEN -29.5 AND -29.0 AND LONGITUDE BETWEEN 30.0 AND 31.0 THEN 'Mooi River'
        WHEN LATITUDE BETWEEN -29.5 AND -28.0 AND LONGITUDE BETWEEN 29.5 AND 31.5 THEN 'Tugela River (Extended)'
        WHEN LATITUDE BETWEEN -28.5 AND -27.5 AND LONGITUDE BETWEEN 31.5 AND 32.5 THEN 'Umfolozi River'
        WHEN LATITUDE BETWEEN -27.5 AND -26.5 AND LONGITUDE BETWEEN 31.5 AND 32.5 THEN 'Pongola River'
        
        -- Mpumalanga/Inkomati System
        WHEN LATITUDE BETWEEN -26.0 AND -25.0 AND LONGITUDE BETWEEN 31.0 AND 32.0 THEN 'Komati River'
        WHEN LATITUDE BETWEEN -26.0 AND -25.5 AND LONGITUDE BETWEEN 30.5 AND 31.5 THEN 'Crocodile River (East)'
        WHEN LATITUDE BETWEEN -25.5 AND -25.0 AND LONGITUDE BETWEEN 31.0 AND 32.0 THEN 'Inkomati River System'
        
        -- Free State rivers
        WHEN LATITUDE BETWEEN -30.0 AND -28.5 AND LONGITUDE BETWEEN 27.0 AND 28.5 THEN 'Caledon River'
        WHEN LATITUDE BETWEEN -29.5 AND -29.0 AND LONGITUDE BETWEEN 25.5 AND 26.5 THEN 'Modder River'
        WHEN LATITUDE BETWEEN -30.0 AND -29.0 AND LONGITUDE BETWEEN 24.5 AND 26.0 THEN 'Riet River'
        WHEN LATITUDE BETWEEN -28.5 AND -27.5 AND LONGITUDE BETWEEN 24.0 AND 25.0 THEN 'Harts River'
        
        ELSE 'Other/Coastal'
    END AS RIVER_SYSTEM
FROM LANDSAT_TRAINING
GROUP BY LATITUDE, LONGITUDE;

In [ ]:
%%sql -r dataframe_5
ALTER TABLE EY_DATA_CHALLENGE.DATA_SCHEMA.WATERQUALITY_TRAINING 
ADD COLUMN RIVER_SYSTEM VARCHAR(100);

-- Update with river classifications
UPDATE EY_DATA_CHALLENGE.DATA_SCHEMA.WATERQUALITY_TRAINING wq
SET RIVER_SYSTEM = rl.RIVER_SYSTEM
FROM EY_DATA_CHALLENGE.DATA_SCHEMA.RIVER_LOCATIONS rl
WHERE wq.LATITUDE = rl.LATITUDE 
  AND wq.LONGITUDE = rl.LONGITUDE;


In [ ]:
%%sql -r dataframe_10
-- Add the column
ALTER TABLE EY_DATA_CHALLENGE.DATA_SCHEMA.LANDSAT_TRAINING 
ADD COLUMN RIVER_SYSTEM VARCHAR(100);

-- Update with river classifications
UPDATE EY_DATA_CHALLENGE.DATA_SCHEMA.LANDSAT_TRAINING lt
SET RIVER_SYSTEM = rl.RIVER_SYSTEM
FROM EY_DATA_CHALLENGE.DATA_SCHEMA.RIVER_LOCATIONS rl
WHERE lt.LATITUDE = rl.LATITUDE 
  AND lt.LONGITUDE = rl.LONGITUDE;

In [ ]:
%%sql -r dataframe_8
select * from LANDSAT_TRAINING where SAMPLE_DATE < '2015-01-01';

In [ ]:
%%sql -r dataframe_9
